In [1]:
pip install yfinance #--target=C:\Users\jazbi\Documents\uzh\3.year\DIg tools for Fin



SyntaxError: invalid syntax (3154460117.py, line 1)

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [31]:
# here specify the number of ETFs held long each period in the text box
w_nLong = widgets.BoundedIntText(
    value=3,
    min=0,
    max=8,
    step=1,
    description='Longs:',
    disabled=False
)
display(w_nLong)

BoundedIntText(value=3, description='Longs:', max=8)

In [32]:
# here specify the number of ETFs held Short each period in the text box (for long-only strategy specify 0)
w_nShort = widgets.BoundedIntText(
    value=3,
    min=0,
    max=8,
    step=1,
    description='Shorts:',
    disabled=False
)
display(w_nShort)

BoundedIntText(value=3, description='Shorts:', max=8)

In [35]:
# here choose level of transaction costs by moving the toggle 
w_tCost = widgets.FloatSlider(
    value=0.001,
    min=0,
    max=0.005,
    step=0.001,
    description='Transaction costs:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
display(w_tCost)

FloatSlider(value=0.001, continuous_update=False, description='Transaction costs:', max=0.005, readout_format=…

In [52]:
# here specify the number of days of trading lag
w_lag = widgets.BoundedIntText(
    value=1,
    min=0,
    max=7,
    step=1,
    description='Lag:',
    disabled=False
)
display(w_lag)

BoundedIntText(value=1, description='Lag:', max=7)

In [53]:
# Parameter selection
# Annualization factor from monthly to annual
annualizationFactor = 12;
# Number of countries held long and short (in Long-only version set nShort to 0)
nLong = int(w_nLong.value)
nShort = int(w_nShort.value)
# Proportional transaction costs
tCost = float(w_tCost.value)
# Momentum lookback period (start and end of window, as positive numbers; start must be greater than end)
lookbackStart = 12;
lookbackEnd = 1;
#trading lag in days
lag = int(w_lag.value)

In [54]:
nLong

5

In [55]:
nShort

0

In [56]:
tCost

0.002

In [57]:
lag

4

In [ ]:
ETFPrices = yf.download(['AAPL', 'F'], 
                   start='2010-01-01', 
                   end='2024-09-20')
ETFPrices

In [ ]:
Rf_Daily = yf.download(['^IRX'], 
                   start='2010-01-01', 
                   end='2024-09-20')

In [ ]:
def get_first_and_last_day_in_period(date_list, n_digits):
    """
    Generate arrays listing the first and last observation in each period.
    Dates should be provided as numeric in the format YYYYMMDD, YYMMDD, or MMDD.
    The granularity of the period is defined by the number of digits that
    are removed from the date list. 2 digits will convert daily data to monthly data
    or monthly data to annual data. 4 digits will convert daily data to annual data.

     date_list: List or numpy array of dates in numeric format 
     n_digits: Number of digits to be removed from the date list to define the period e.g. two to tet months
    :return: Two lists containing the indices of the first and last observations in each period
    """
    n_obs = len(date_list)
    scaling_factor = 1 / (10 ** n_digits)
    trimmed_date = np.round(date_list * scaling_factor).astype(int)
    last_day_of_period = trimmed_date[:-1] - trimmed_date[1:]

    # Generate a list with the last days, and use it to generate the list of the first days
    last_day_list = np.where(last_day_of_period != 0)[0]
    first_day_list = last_day_list + 1
    last_day_list = np.append(last_day_list, n_obs - 1)  # Add the last day in the sample
    first_day_list = np.insert(first_day_list, 0, 0)     # Add the first day in the sample

    return first_day_list, last_day_list

In [ ]:
#get nMonths and nAssets
last_days_list_Months = get_first_and_last_day_in_period(ETFPrices, 2)['last_day_list']#see hwat the actual output of the function is
nMonths = size(last_days_list,1) - 1
nAssets = size(ETF, 2)

In [ ]:
#get returns from prices -- still daily
ETFReturns_Daily = np.zeros(nMonths, nAssets)
ETFReturns_Daily[1:, :] = ETFPrices[1:, :] / ETFPrices[:-1, :] - 1

In [ ]:
# Compute monthlyDates4Fig using dates4Fig function
dates4Fig = ETFPrices['Date']
monthlyDates4Fig = dates4Fig(lastDayList)
# Assuming dates4Fig is a pandas Series or list of dates in string format
# Convert dates4Fig to a pandas datetime format
dates4Fig = pd.to_datetime(dates4Fig)
# Convert the datetime to a numeric format (number of days since a reference date, e.g., 1970-01-01)
datesNumeric = (dates4Fig - pd.Timestamp("1970-01-01")) / pd.Timedelta(days=1)
datesNumeric = datesNumeric.to_numpy()
##########################THIS MIGHT BE WRONG IDK HOW CHATGPT DID THIS #######################################
# Convert to YYYYMMDD format
dates = dates4Fig.dt.strftime('%Y%m%d')#.astype(int)  # Convert to int if needed

In [ ]:
#function for weights computation
def compute_sort_weights(sort_variable, n_longs, n_shorts, long_high_values):
    """
    Generates portfolio weights based on sort_variable.
    
    Parameters:
    - sort_variable: Array of values used to sort assets.
    - n_longs: Number of assets to hold long.
    - n_shorts: Number of assets to hold short.
    - long_high_values: If 1, assets with the highest values are held long.
    
    Returns:
    - weights: Array of weights assigned to each asset.
    """
    # Remove NaN values and get indices
    sort_variable = np.array(sort_variable)
    n_assets = len(sort_variable)
    
    # Initialize weights
    weights = np.zeros(n_assets)
    
    # Handle NaN values by using boolean indexing
    valid_indices = ~np.isnan(sort_variable)
    valid_sort_variable = sort_variable[valid_indices]
    
    if long_high_values == 1:
        # Get indices of highest and lowest values
        list_of_longs = np.argsort(valid_sort_variable)[-n_longs:]  # Highest
        list_of_shorts = np.argsort(valid_sort_variable)[:n_shorts]  # Lowest
    else:
        list_of_longs = np.argsort(valid_sort_variable)[:n_longs]  # Lowest
        list_of_shorts = np.argsort(valid_sort_variable)[-n_shorts:]  # Highest
    
    # Assign weights
    weights[valid_indices] = 0  # Reset weights for valid indices
    weights[valid_indices][list_of_longs] = 1 / n_longs
    weights[valid_indices][list_of_shorts] = -1 / n_shorts

    return weights

In [ ]:
ETFReturns_Monthly = np.zeros((nMonths, nAssets))
momLSWeights = np.zeros((nMonths, nAssets))
# Second, compute the return in the month, honoring any trading lag, 
# and obtain the portfolio weights by sorting on past returns.
for m in range(nMonths):
    first = max(firstDayList[m] - lag, 0)  
    last = lastDayList[m] - lag
    
    # Calculate the cumulative product of returns over the period
    ETFReturns_Monthly = np.prod(1 + ETFReturns_Daily[first:last, :], axis=0)
    
    # Compute weights using the computeSortWeights function
    momLSWeights[m, :] = computeSortWeights(monthlyRet, nLongs, nShorts, 1)
# In order to have weights and returns in sync, one drops firstMonth months from the beginning of the return series, and (firstMonth - 1) months from 
# the beginning of the portfolio weight series. We keep one extra month at the end of the portfolio weights series for the turnover computations later.
#this is TBDone but I first wanted dthe sceleton of the code before running with data

In [ ]:
# Assuming dateNumeric and Rf are defined as numpy arrays

# Compute the number of calendar days until the end of the next month
dayCount = np.diff(datesNumeric)

# Initialize RfMonthly as a zeros array
RfMonthly = np.zeros((nMonths, 1))

# Calculate the monthly returns for the riskless asset, shifted down by one month
RfMonthly[1:, 0] = Rf[:-1, 0] * dayCount / 360 

In [ ]:
# Compute strategy returns without transaction costs
# Replace NaN returns with zeros
ETFReturns_Monthly = np.nan_to_num(ETFReturns_Monthly)  # This replaces NaN with 0

# Initialize an array for strategy returns
stratReturnsNoTC = np.zeros((nMonths, 2))

# Compute returns for equal weights, momentum long weights, and momentum long-short weights
stratReturnsNoTC[:, 1] = np.sum(ETFReturns_Monthly[:-1, :] * momLSWeights[:-1, :], axis=1) + RfMonthly#still need to define the monthly Rf return
stratReturnsNoTC[:, 2] = np.sum(ETFReturns_Monthly[:-1, :] * momLSWeights[:-1, :], axis=1) 

In [ ]:
#calcilate the startegy cumulative return for the graph of strategy performance
# Assuming allTotalReturns is a numpy array 
strategyNAV = np.cumprod(1 + stratReturnsNoTC)

In [ ]:
#graph of strategy performance
# Assuming dates4Fig is a pandas Series or a NumPy array of dates
# and strategyNAV is a NumPy array
plt.figure(figsize=(8, 6))

# Plotting the strategy NAV for different portfolios
plt.plot(dates4Fig, strategyNAV[:, 1], 'b-', label='Momentum Long Only')
plt.plot(dates4Fig, strategyNAV[:, 2], 'r-', label='Momentum Long/Short')

# Labeling axes
plt.xlabel('Year')
plt.ylabel('Portfolio Value')

# Adding legend
plt.legend(loc='upper left')

# Setting font size
plt.tick_params(axis='both', labelsize=15)

# Show the plot
plt.tight_layout()
plt.show()